<a href="https://colab.research.google.com/github/ncn-foreigners/job-ads-classifier/blob/main/job_classification_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Colab settings

In [15]:
from google.colab.data_table import DataTable
DataTable.max_columns = 60

In [1]:
!sudo apt-get install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.


In [8]:
!git lfs install

In [9]:
!git clone https://github.com/ncn-foreigners/job-ads-classifier.git 

Cloning into 'job-ads-classifier'...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 101 (delta 32), reused 45 (delta 5), pack-reused 0
Receiving objects: 100% (101/101), 2.16 MiB | 4.55 MiB/s, done.
Resolving deltas: 100% (32/32), done.
tcmalloc: large alloc 1471086592 bytes == 0x56087dc86000 @  0x7f83bfa922a4 0x56084239511f 0x56084237225b 0x560842326f33 0x5608422cb22a 0x5608422cb6e6 0x5608422e8451 0x5608422e89e9 0x5608422e8f13 0x56084238de12 0x56084222f162 0x560842215a65 0x560842216725 0x56084221572a 0x7f83bedd9c87 0x56084221577a
tcmalloc: large alloc 2206621696 bytes == 0x5608d5776000 @  0x7f83bfa922a4 0x56084239511f 0x56084237225b 0x560842326f33 0x5608422cb22a 0x5608422cb6e6 0x5608422e8451 0x5608422e89e9 0x5608422e8f13 0x56084238de12 0x56084222f162 0x560842215a65 0x560842216725 0x56084221572a 0x7f83bedd9c87 0x56084221577a
Filtering content: 100% (14/14), 2.09 GiB | 6.70 MiB/s, don

In [12]:
!python -m pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 13.2 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3


In [13]:
!pip install virtualenv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 79.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 kB 39.7 MB/s eta 0:00:00


In [14]:
!virtualenv classifier

created virtual environment CPython3.7.15.final.0-64 in 680ms
  creator CPython3Posix(dest=/content/classifier, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==22.2.2, setuptools==65.3.0, wheel==0.37.1
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [16]:
!source /content/classifier/bin/activate

## Set up environment

Install required packages

In [ ]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
!pip install utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
!pip install -r /content/job-ads-classifier/requirements-py37-colab.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.5/541.5 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 65.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.7/361.7 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 708.1/708.1 kB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.4/660.4 kB 46.1 MB/s eta 0:00:00
  

## Working with trained models

In [1]:
models_dir="/content/job-ads-classifier/models"

Load required modules

In [2]:
import sys
sys.path.insert(1, '/content/job-ads-classifier')

In [7]:
from utils import *

In [3]:
# Numpy, pandas i matplotlib jako pomocnicze biblioteki
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from job_offers_classifier.load_save import load_to_df, save_obj, load_obj
from job_offers_classifier.job_offers_utils import create_hierarchy, fix_class_str, remove_classes, filter_classes, remap_classes
from job_offers_classifier.job_offers_classfier import *
from sklearn.model_selection import train_test_split
import torch
from torchmetrics import MetricCollection, Recall, Accuracy, ConfusionMatrix

Example data -- job offers from Public Sercice

In [10]:
kprm_df=pd.read_xml("https://nabory.kprm.gov.pl/pls/serwis/app.xml", xpath = "./new/oferta")
kprm_df = kprm_df.replace("\\<\\!\\[CDATA\\[ ", "", regex=True)
kprm_df = kprm_df.replace(" \\]\\]\\>", "", regex=True)
kprm_df = kprm_df.replace(r'<[^<>]*>', '', regex=True)
kprm_df = kprm_df.replace('&nbsp;', ' ', regex=True)
kprm_df = kprm_df.replace('&dash;', ' ', regex=True)
kprm_df = kprm_df.replace('&oacute;', ' ', regex=True)
kprm_df = kprm_df.replace('\r', ' ', regex=True)
kprm_df['desc'] = kprm_df[['stanowisko', 'do_spraw', 'nazwa_firmy', 'poledodatkowelista1', 'poledodatkowelista3']].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
kprm_df[["unid", "nazwa_firmy", "stanowisko", "do_spraw", "poledodatkowelista1", "poledodatkowelista3"]].head(n=2)


,unid,nazwa_firmy,stanowisko,do_spraw,poledodatkowelista1,poledodatkowelista3
0,108581,Ministerstwo Spraw Zagranicznych w Warszawie,inspektor,ekspedycji informacji niejawnych,Prowadzi dzienniki ewidencji materiałów niejaw...,Warunki dotyczące charakteru pracy na stanowis...
1,108580,Powiatowy Inspektorat Nadzoru Budowlanego w Op...,inspektor nadzoru budowlanego,kontroli,Przygotowywanie i opracowywanie projektów decy...,Praca w siedzibie Inspektoratu przy wykorzysta...


In [18]:
kprm_df.desc[0]

'inspektor_ekspedycji informacji niejawnych_Ministerstwo Spraw Zagranicznych w Warszawie_Prowadzi dzienniki ewidencji materiałów niejawnych.Rejestruje wpływające pilne notatki, notatki informacyjne, szyfrogramy.Opracowuje w ujednoliconej graficznie postaci „papierowej” nadesłane  przez placówki zagraniczne szyfrogramy oraz przygotowuje je do  ekspediowania według stopnia pilności, zgodnie z rozdzielnikiem.Przekazuje w systemie teleinformatycznym otrzymane z placówek zagranicznych szyfrogramy do adresatów w MSZ. Prowadzi obieg informacji niejawnych przetwarzanych w systemach teleinformatycznych przeznaczonych do przetwarzania informacji niejawnych.Wykonuje kopie dokumentów niejawnych (pilne notatki i notatki informacyjne).Przygotowuje i przekazuje do archiwum egzemplarze aktowe szyfrogramów._Warunki dotyczące charakteru pracy na stanowisku i sposobu wykonywania zadań:1.       Nietypowe godziny pracy (w tym dyżury). 2.       Zagrożenie naciskami grup przestępczych. 3.       Częste reprez

### Logistic regression model using tf-df matrix

In [12]:
model_paper_linear = LinearJobOffersClassifier()
model_paper_linear.load(f"{models_dir}/linear_model")

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:333: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.23.2 when using version 1.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:333: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.23.2 when using version 1.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


In [13]:
kprm_df_zaw1 = model_paper_linear.predict(
    X_text=list(kprm_df["poledodatkowelista1"]),
    output_level=0,
    format="dataframe"
)

Processing X ...
Processing text ...


100%|██████████| 450/450 [00:01<00:00, 339.55it/s]


Transforming text to tf-idf ...
Predicting ...


100%|██████████| 450/450 [00:00<00:00, 1630.23it/s]


In [15]:
np.sum(kprm_df_zaw1, axis=0)

0      0.048865
1     28.085716
2    198.205139
3    178.860062
4     30.369036
5      5.723581
6      0.961112
7      3.962847
8      2.878015
9      0.885257
dtype: float32

In [16]:
kprm_df_zaw6_top5 = model_paper_linear.predict(
    X_text=list(kprm_df["poledodatkowelista1"]),
    output_level="last",
    format="dataframe",
    top_k=5
)

Processing X ...
Processing text ...


100%|██████████| 450/450 [00:01<00:00, 347.59it/s]


Transforming text to tf-idf ...
Predicting ...


100%|██████████| 450/450 [00:00<00:00, 1632.41it/s]


In [17]:
kprm_df_zaw6_top5.head(n=5)

,class_1,class_2,class_3,class_4,class_5,prob_1,prob_2,prob_3,prob_4,prob_5
0,242110,334102,334101,334306,243390,0.218574,0.155126,0.039867,0.029361,0.028181
1,242211,311211,214202,311204,311212,0.991834,0.003184,0.001015,0.000571,0.000427
2,242217,241203,242211,242223,241290,0.071343,0.064142,0.044098,0.037160,0.036222
3,325512,225290,225211,225209,314204,0.367114,0.063721,0.049297,0.027435,0.023879
4,325512,225211,225290,225207,225101,0.350336,0.331611,0.058897,0.019308,0.018973


### Transformer model

In [20]:
model_paper_trans = TransformerJobOffersClassifier(gpus=0, threads=5)
model_paper_trans.load(f"{models_dir}/transformer_model")

Initializing TransformerClassifier with model_name=allegro/herbert-base-cased, output_type=linear, num_labels=2905, learning_rate=1e-05, weight_decay=0.01, warmup_steps=50 ...


Downloading:   0%|          | 0.00/624M [00:00<?, ?B/s]

Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.sso.sso_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.sso.sso_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
pracuj_kod2 = model_paper_trans.predict(
    X=list(kprm_df["poledodatkowelista1"]),
    output_level=0,
    format="dataframe"
)